In [2]:
# ===============================================
# Final Version: Proposal-Aligned with Secondary Metrics
# - Primary Metric: Mismatch Rate (as per proposal)
# - Secondary Metrics: Correlation, MAE, RMSE (for deeper analysis)
# - Core Logic: Based on proposal (Regression, MSE, No Extras)
# - ADDED: Real-time progress bar using tqdm
# - OPTIMIZED: Faster I/O by copying data locally
# ===============================================

!pip -q install transformers scipy tqdm

import os, math, random, numpy as np, pandas as pd, torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from scipy.stats import pearsonr, spearmanr
from tqdm.auto import tqdm # 실시간 진행률 표시를 위해 tqdm 추가

# ----- 0) Paths / Device / Seed -----
from google.colab import drive
drive.mount('/content/drive')

# OPTIMIZATION: Copy data from Drive to the local Colab VM for much faster I/O
# 구글 드라이브에서 로컬로 파일을 복사하여 데이터 로딩 속도를 향상시킵니다.
print("Copying data file from Google Drive to local Colab VM...")
!cp "/content/drive/MyDrive/Colab Notebooks/sampled_400_sentence_level_no_dedup_latest.xlsx" "/content/data.xlsx"
print("Copy complete.")

FILE_PATH  = "/content/data.xlsx"  # 이제 훨씬 빠른 로컬 파일을 사용합니다.
SHEET_NAME = "sentences"
OUT_DIR    = "/content/drive/MyDrive/sentiment_outputs_final2"
os.makedirs(OUT_DIR, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
seed = 42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if device=="cuda":
    torch.cuda.manual_seed_all(seed)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Copying data file from Google Drive to local Colab VM...
Copy complete.


In [3]:
# ----- 1) Load & Create Labels (rating -> s_i) -----
df = pd.read_excel(FILE_PATH, sheet_name=SHEET_NAME)
need = {"review_seq_id","sentence_text","rating"}
missing = need - set(df.columns)
assert not missing, f"필수 컬럼 누락: {missing}"

df["rating"] = pd.to_numeric(df["rating"], errors="coerce").round().clip(1,5)
df = df.dropna(subset=["sentence_text","rating"]).copy()

def rating_to_si(r):
    return (r - 3.0) / 2.0
df["label_si"] = df["rating"].apply(rating_to_si).astype(np.float32)

if "sentence_index" not in df.columns:
    df["sentence_index"] = df.groupby("review_seq_id").cumcount()
if "n_sentences_in_review" not in df.columns:
    df["n_sentences_in_review"] = df.groupby("review_seq_id")["sentence_text"].transform("size")

# ----- 2) Split by review_seq_id to prevent leakage -----
uids = df["review_seq_id"].dropna().unique()
rng = np.random.default_rng(seed); rng.shuffle(uids)
val_ratio = 0.1
n_val = int(len(uids)*val_ratio)
val_ids = set(uids[:n_val])
train = df[~df["review_seq_id"].isin(val_ids)].reset_index(drop=True)
valid = df[ df["review_seq_id"].isin(val_ids)].reset_index(drop=True)
print(f"[Split] train={len(train):,} sentences | valid={len(valid):,} sentences")


# ----- 3) Tokenizer / Dataset -----
model_name = "bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_len = 128
class SentDS(Dataset):
    def __init__(self, pdf):
        self.texts = pdf["sentence_text"].astype(str).tolist()
        self.labels = pdf["label_si"].astype(np.float32).values
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = tokenizer(self.texts[i], padding="max_length", truncation=True,
                        max_length=max_len, return_tensors="pt")
        item = {k:v.squeeze(0) for k,v in enc.items()}
        item["labels"] = torch.tensor(self.labels[i], dtype=torch.float32)
        return item

ds_tr, ds_va = SentDS(train), SentDS(valid)
# OPTIMIZATION: num_workers를 시스템 권장 사항인 2로 조정합니다.
tr_loader = DataLoader(ds_tr, batch_size=16, shuffle=True,  num_workers=2, pin_memory=True)
va_loader = DataLoader(ds_va, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)


[Split] train=62,717 sentences | valid=7,240 sentences


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [4]:
# ----- 4) Model Training (Proposal-Based: Regression, MSE) -----
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=1, problem_type="regression"
).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
mse = torch.nn.MSELoss()

def loss_fn(logits, labels):
    preds = torch.tanh(logits.squeeze(-1))
    return mse(preds, labels), preds

epochs = 3
for ep in range(1, epochs+1):
    model.train(); tr_loss = 0.0
    progress_bar = tqdm(tr_loader, desc=f"Epoch {ep} Training", leave=False)
    for step, batch in enumerate(progress_bar):
        batch = {k:v.to(device) for k,v in batch.items()}
        labels = batch.pop("labels")
        optimizer.zero_grad(set_to_none=True)
        out = model(**batch)
        loss, _ = loss_fn(out.logits, labels)
        loss.backward()
        optimizer.step()
        tr_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    tr_loss /= (step + 1)

    model.eval(); va_loss = 0.0
    val_progress_bar = tqdm(va_loader, desc=f"Epoch {ep} Validation", leave=False)
    with torch.no_grad():
        for batch in val_progress_bar:
            batch = {k:v.to(device) for k,v in batch.items()}
            labels = batch.pop("labels")
            out = model(**batch)
            l, _ = loss_fn(out.logits, labels)
            va_loss += l.item()

    va_loss /= len(va_loader)
    print(f"[Epoch {ep}] train_loss={tr_loss:.4f} | valid_loss={va_loss:.4f}")

# Save the final model after 3 epochs
save_dir = os.path.join(OUT_DIR, "bert_reg_final_model")
model.save_pretrained(save_dir); tokenizer.save_pretrained(save_dir)
print(">> Saved final model to:", save_dir)


model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 Training:   0%|          | 0/3920 [00:00<?, ?it/s]

Epoch 1 Validation:   0%|          | 0/227 [00:00<?, ?it/s]

[Epoch 1] train_loss=0.3159 | valid_loss=0.2706


...

Epoch 2 Validation:   0%|          | 0/227 [00:00<?, ?it/s]

[Epoch 2] train_loss=0.2412 | valid_loss=0.2555


Epoch 3 Training:   0%|          | 0/3920 [00:00<?, ?it/s]

Epoch 3 Validation:   0%|          | 0/227 [00:00<?, ?it/s]

[Epoch 3] train_loss=0.2035 | valid_loss=0.2610
>> Saved final model to: /content/drive/MyDrive/sentiment_outputs_final2/bert_reg_final_model


In [6]:
# ----- 5) Full Inference to get sentence-level s_i -----
tok = AutoTokenizer.from_pretrained(save_dir)
mdl = AutoModelForSequenceClassification.from_pretrained(
    save_dir, num_labels=1, problem_type="regression"
).to(device).eval()

class InferDS(Dataset):
    def __init__(self, pdf):
        self.texts = pdf["sentence_text"].astype(str).tolist()
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = tok(self.texts[i], padding="max_length", truncation=True,
                  max_length=max_len, return_tensors="pt")
        return {k:v.squeeze(0) for k,v in enc.items()}

inf_loader = DataLoader(InferDS(df), batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
preds = np.empty(len(df), dtype=np.float32); off=0

inference_progress_bar = tqdm(inf_loader, desc="Inferencing", leave=True)
with torch.no_grad():
    for batch in inference_progress_bar:
        batch = {k:v.to(device) for k,v in batch.items()}
        out = mdl(**batch)
        si = torch.tanh(out.logits.squeeze(-1)).detach().cpu().numpy().astype(np.float32)
        preds[off:off+len(si)] = si; off += len(si)

df["sentiment_score"]  = preds
def label_from_score(s, thr=0.05):
    if s >=  thr: return "positive"
    if s <= -thr: return "negative"
    return "neutral"
# [FIX] sentiment_label -> sentiment_score
df["sentiment_label"] = [label_from_score(x) for x in df["sentiment_score"]]


Inferencing:   0%|          | 0/2187 [00:00<?, ?it/s]

In [7]:
# ----- 6) Review-level aggregation using the proposal rule -----
df["is_boundary"] = ((df["sentence_index"]==0) |
                      (df["n_sentences_in_review"]==df["sentence_index"]+1)).astype(int)
alpha, beta = 1.0, 1.0
df["weight"] = alpha*df["sentiment_score"].abs() + beta*df["is_boundary"]

def wavg(g):
    w = g["weight"].to_numpy(float)
    s = g["sentiment_score"].to_numpy(float)
    if np.isfinite(w).any() and w.sum() > 0:
        return np.average(s, weights=w)
    else:
        return float(np.nanmean(s))

wavg_df = (df[["review_seq_id","weight","sentiment_score"]]
           .groupby("review_seq_id").apply(wavg).rename("sentiment_score_w").reset_index())

meta_cols = [c for c in ["rating","brand_name","submission_time"] if c in df.columns]
meta = (df.sort_values("submission_time") if "submission_time" in df.columns else df).drop_duplicates("review_seq_id")
meta = meta[["review_seq_id"] + meta_cols] if meta_cols else meta[["review_seq_id"]]
review_w = wavg_df.merge(meta, on="review_seq_id", how="left")
# [FIX] sentiment_label_w -> sentiment_score_w
review_w["sentiment_label_w"] = [label_from_score(x) for x in review_w["sentiment_score_w"]]


/tmp/ipython-input-1121357853.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .groupby("review_seq_id").apply(wavg).rename("sentiment_score_w").reset_index())


In [8]:
# ----- 7) Evaluation: Primary (Proposal) + Secondary (Deeper Analysis) -----
def score_to_star_bucket(s):
    if s <= -0.6: return 1
    elif s <= -0.3: return 2
    elif s <  0.3:  return 3
    elif s <= 0.6:  return 4
    else:           return 5

ev = review_w.dropna(subset=["rating","sentiment_score_w"]).copy()
if ev.empty:
    print("\n[Eval] No ratings available. Skipping evaluation.")
else:
    print("\n--- Primary Metric (As per Proposal) ---")
    ev["pred_star_bucket"] = ev["sentiment_score_w"].apply(score_to_star_bucket).astype(int)
    ev["rating"] = ev["rating"].astype(int)
    ev["match"] = (ev["pred_star_bucket"] == ev["rating"]).astype(int)
    mismatch_rate = 1.0 - ev["match"].mean()
    print(f"Mismatch Rate: {mismatch_rate:.3f}")

    ct = pd.crosstab(ev["rating"], ev["pred_star_bucket"],
                       rownames=["True Star"], colnames=["Pred Star"], dropna=False)
    print("\n[Confusion Matrix]\n", ct.to_string())

    print("\n--- Secondary Metrics (For Deeper Analysis) ---")
    pearson_corr = pearsonr(ev["rating"], ev["sentiment_score_w"])[0]
    spearman_corr = spearmanr(ev["rating"], ev["sentiment_score_w"])[0]

    pred_star_continuous = 3 + 2 * ev["sentiment_score_w"]
    mae = float(np.mean(np.abs(pred_star_continuous - ev["rating"])))
    rmse = float(np.sqrt(np.mean((pred_star_continuous - ev["rating"])**2)))

    print(f"Pearson Correlation : {pearson_corr:.3f}")
    print(f"Spearman Correlation: {spearman_corr:.3f}")
    print(f"MAE (on 1-5 scale)  : {mae:.3f}")
    print(f"RMSE (on 1-5 scale) : {rmse:.3f}")



--- Primary Metric (As per Proposal) ---
Mismatch Rate: 0.315

[Confusion Matrix]
 Pred Star    1    2     3    4     5
True Star                           
1          905  703   221   10     3
2          300  936   525   36     8
3           50  481  1133  324    89
4            1   21   310  749  1901
5            0    1    42  259  7792

--- Secondary Metrics (For Deeper Analysis) ---
Pearson Correlation : 0.914
Spearman Correlation: 0.847
MAE (on 1-5 scale)  : 0.468
RMSE (on 1-5 scale) : 0.591


In [9]:
# ----- 8) Save final artifacts for next stages -----
sent_csv = os.path.join(OUT_DIR, "sentence_level_sentiment.csv")
revw_csv = os.path.join(OUT_DIR, "review_level_sentiment.csv")
df.to_csv(sent_csv, index=False); review_w.to_csv(revw_csv, index=False)
print(f"\n[Saved] Sentence-level results: {sent_csv}")
print(f"[Saved] Review-level results: {revw_csv}")


[Saved] Sentence-level results: /content/drive/MyDrive/sentiment_outputs_final2/sentence_level_sentiment.csv
[Saved] Review-level results: /content/drive/MyDrive/sentiment_outputs_final2/review_level_sentiment.csv
